## Visualise stokes annulus model (flow etc)

In [1]:
# to fix trame issue
import nest_asyncio
nest_asyncio.apply()
import os

In [2]:
import petsc4py
import underworld3 as uw
import numpy as np
import sympy

[esdhcp-137.anu.edu.au:24069] shmem: mmap: an error occurred while determining whether or not /var/folders/tx/95gr762j29z4tt5d1dnqlgth0000gn/T//ompi.esdhcp-137.501/jf.0/2649161728/sm_segment.esdhcp-137.501.9de70000.0 could be created.


In [3]:
ls -trl output/annulus/ | tail

total 0
drwxr-xr-x  3233 lmoresi  staff  103456 Aug 15 10:37 Ra1e5/
drwxr-xr-x     2 lmoresi  staff      64 Aug 15 20:26 Ra1e4.0/
drwxr-xr-x@  804 lmoresi  staff   25728 Aug 16 13:18 Ra1e5.0/
drwxr-xr-x  2271 lmoresi  staff   72672 Aug 18 22:32 Ra1e6.0/


In [4]:
ls -trl output/annulus/Ra1e6.0/ | tail

-rw-r--r--    1 lmoresi  staff    3090 Aug 18 22:32 annulus_02_Ra1e6.0_res20.mesh.00297.xdmf
-rw-r--r--    1 lmoresi  staff  258744 Aug 18 22:32 annulus_02_Ra1e6.0_res20.mesh.U.00298.h5
-rw-r--r--    1 lmoresi  staff   88408 Aug 18 22:32 annulus_02_Ra1e6.0_res20.mesh.P.00298.h5
-rw-r--r--    1 lmoresi  staff  453176 Aug 18 22:32 annulus_02_Ra1e6.0_res20.mesh.T.00298.h5
-rw-r--r--    1 lmoresi  staff    3090 Aug 18 22:32 annulus_02_Ra1e6.0_res20.mesh.00298.xdmf
-rw-r--r--    1 lmoresi  staff  258744 Aug 18 22:32 annulus_02_Ra1e6.0_res20.mesh.U.00299.h5
-rw-r--r--    1 lmoresi  staff   88408 Aug 18 22:32 annulus_02_Ra1e6.0_res20.mesh.P.00299.h5
-rw-r--r--    1 lmoresi  staff  453176 Aug 18 22:32 annulus_02_Ra1e6.0_res20.mesh.T.00299.h5
-rw-r--r--    1 lmoresi  staff    3090 Aug 18 22:32 annulus_02_Ra1e6.0_res20.mesh.00299.xdmf
drwxr-xr-x@ 504 lmoresi  staff   16128 Aug 18 22:33 images/


In [5]:
geometry = "annulus_02"
resolution = 15

checkpoint_dir = f"output/{geometry}/Ra1e6.0"
checkpoint_base = f"{geometry}_Ra1e6.0_res{resolution}"
meshfile = os.path.join(checkpoint_dir, checkpoint_base) + ".mesh.00000.h5"
image_dir = os.path.join(checkpoint_dir,"images")
os.makedirs(image_dir, exist_ok=True)

In [6]:
mesh = uw.discretisation.Mesh(meshfile, 
                              coordinate_system_type=uw.coordinates.CoordinateSystemType.CYLINDRICAL2D)
    
x = mesh.N.x
y = mesh.N.y

v_soln = uw.discretisation.MeshVariable("U", mesh, mesh.dim, degree=2)
t_soln = uw.discretisation.MeshVariable("T", mesh, 1, degree=3)
flux = uw.discretisation.MeshVariable(r"dTdz", mesh, 1, degree=1)

[0]PETSC ERROR: --------------------- Error Message --------------------------------------------------------------
[0]PETSC ERROR: Error in external library
[0]PETSC ERROR: Error in HDF5 call H5Fopen() Status -1
[0]PETSC ERROR: WARNING! There are unused option(s) set! Could be the program crashed before usage or a spelling mistake, etc!
[0]PETSC ERROR:   Option left: name:-f value: /Users/lmoresi/Library/Jupyter/runtime/kernel-94bff0bc-3d40-452b-9245-24b16222126b.json source: command line
[0]PETSC ERROR: See https://petsc.org/release/faq/ for trouble shooting.
[0]PETSC ERROR: Petsc Development GIT revision: v3.21.4-437-gbf0460ce64e  GIT Date: 2024-08-14 09:06:37 -0500
[0]PETSC ERROR: /Users/lmoresi/mambaforge/envs/underworld3p/lib/python3.11/site-packages/ipykernel_launcher.py with 1 MPI process(es) and PETSC_ARCH arch-darwin-c-opt on esdhcp-137.anu.edu.au by lmoresi Wed Aug 21 10:25:36 2024
[0]PETSC ERROR: Configure options: --download-eigen=1 --download-hdf5=0 --download-metis=1 --do

Error: error code 76

Exception ignored in: 'petsc4py.PETSc.Object.__dealloc__'
Traceback (most recent call last):
  File "/Users/lmoresi/mambaforge/envs/underworld3p/lib/python3.11/site-packages/underworld3/discretisation.py", line 98, in _from_plexh5
    viewer = PETSc.ViewerHDF5().create(filename, "r", comm=comm)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
petsc4py.PETSc.Error: error code 76


Error: error code 76

In [ ]:
steps = range(140,150,1)

In [ ]:
import mpi4py
import pyvista as pv
import underworld3.visualisation as vis

pl = pv.Plotter(window_size=[1000, 500])

for step in steps:

    try:
        v_soln.read_timestep(checkpoint_base, "U", step, outputPath=checkpoint_dir)
        t_soln.read_timestep(checkpoint_base, "T", step, outputPath=checkpoint_dir)
        print(f"Plotting step {step}")

    except:
        print(f"Failed to read step {step} correctly")
        continue

    pl.clear()

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    # pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, t_soln.sym[0])
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)

    pvmesh_v = vis.meshVariable_to_pv_mesh_object(v_soln, alpha=None)
    pvmesh_t = vis.meshVariable_to_pv_mesh_object(t_soln, alpha=None)

    pvmesh_v.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh_v, v_soln.sym)
    pvmesh_t.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh_t, t_soln.sym)
    
    # point sources at cell centres
    skip = 10
    points = np.zeros((mesh._centroids[::skip].shape[0], 3))
    points[:, 0] = mesh._centroids[::skip, 0]
    points[:, 1] = mesh._centroids[::skip, 1]
    point_cloud = pv.PolyData(points)

    pl.add_mesh(
        pvmesh_t,
        cmap="RdBu_r",
        edge_color="Black",
        edge_opacity=0.1,
        show_edges=True,
        scalars="T",
        use_transparency=False,
        opacity=1.0,
        show_scalar_bar=False
    )

    pvstream = pvmesh.streamlines_from_source(
        point_cloud,
        vectors="V",
        integration_direction="both",
        max_time=1,
        surface_streamlines=True,
    )
    

    pl.add_mesh(pvstream, opacity=0.4, show_scalar_bar=False)    

    # pl.camera_position = [(3.0, 0.5, 6.750994846396983),
    #                       (3.0, 0.5, 0.0),
    #                       (0.0, 1.0, 0.0)]

    imagefile = os.path.join(image_dir, checkpoint_base) + f"_V_T_{step}.png"
    
    pl.screenshot(filename=imagefile,
                  window_size=(1000, 500),
                  return_img=False)



In [ ]:
pl.show(jupyter_backend="trame")


In [ ]:
## Calculate heat flux, evaluate at surface — proxy for boundary layer thickness

In [ ]:
t_soln.stats()

In [ ]:
0/0

In [ ]:
flux_solver = uw.systems.Projection(mesh, flux)

# Conductive flux only !
radial_flux = -mesh.vector.gradient(t_soln.sym[0]).dot(mesh.CoordinateSystem.unit_e_0)
radial_flux *= sympy.exp(-100*(r-1)**2) 

flux_solver.uw_function = radial_flux
flux_solver.smoothing = 1.0e-3
flux_solver.solve()

In [ ]:
import mpi4py
import pyvista as pv
import underworld3.visualisation as vis

pvmesh = vis.mesh_to_pv_mesh(mesh)
pvmesh.point_data["dTdz"] = vis.scalar_fn_to_pv_points(pvmesh, flux.sym[0])
pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)
pvmesh.point_data["V"] -= pvmesh.point_data["V"].mean()

velocity_points = vis.meshVariable_to_pv_cloud(v_soln)
velocity_points.point_data["V"] = vis.vector_fn_to_pv_points(velocity_points, v_soln.sym)

# point sources at cell centres
skip = 8
points = np.zeros((mesh._centroids[::skip].shape[0], 3))
points[:, 0] = mesh._centroids[::skip, 0]
points[:, 1] = mesh._centroids[::skip, 1]
point_cloud = pv.PolyData(points)

pvstream = pvmesh.streamlines_from_source(
    point_cloud,
    vectors="V",
    integration_direction="both",
    max_time=0.5,
)

pl = pv.Plotter(window_size=[1000, 1000])

pl.add_mesh(
    pvmesh,
    cmap="Reds",
    edge_color="Black",
    show_edges=True,
    scalars="dTdz",
    use_transparency=False,
    opacity=1.0,
)

# 
# pl.add_mesh(pvmesh, "Black", "wireframe",  opacity=0.1)

# pl.add_points(point_cloud, color="White", point_size=3.0, opacity=0.25)

# pl.add_arrows(velocity_points.points, velocity_points.point_data["V"], mag=0.0001, show_scalar_bar=True)
pl.add_mesh(pvstream, opacity=0.4, show_scalar_bar=False)
    
# pl.remove_scalar_bar("V")

imagefile = os.path.join(image_dir, checkpoint_base) + f"_dTdr_{step}.png"

pl.screenshot(filename=imagefile, window_size=(750, 750), return_img=False)
# OR


In [ ]:
pl.show()